In [1]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, DatetimeTickFormatter
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps


import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
locale.setlocale(locale.LC_ALL,"Danish")

# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

'Danish_Denmark.1252'

# Load data 

In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd()[:-5] + ssidatapath
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
dfCase = pd.DataFrame(columns=dfAdm.columns)

In [25]:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

In [27]:
dfCase.diff()

Aldersgruppe,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,I alt
10_11_2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17_11_2020,502,1376,1476,1009,1150,1110,550,275,113,32,7593
24_11_2020,550,1607,1545,1137,1235,1208,640,341,165,42,8470
01_12_2020,579,1739,1668,1167,1309,1228,641,396,158,43,8928
08_12_2020,757,2712,2523,1655,1917,1735,781,511,195,64,12850
15_12_2020,1279,3836,4569,2915,3085,2925,1491,746,337,105,21288
22_12_2020,1801,3902,4655,3218,3463,3399,1868,1179,464,139,24088
29_12_2020,1665,2358,3055,2519,2692,2670,1481,1024,604,204,18272
05_01_2021,981,1754,2515,1984,1913,2078,1362,946,603,196,14332
12_01_2021,773,1372,1997,1407,1426,1660,1000,689,510,188,11022


# Data: Collect the admission data in same style

In [56]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]

dfCaseDiff['Dato'] = curDates[1:]

# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.4)

dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

# dfCaseDiff = dfCaseDiff.iloc[:-1]
# dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]


dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']
dfCaseDiffSum

Aldersgruppe,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Dato,I alt
17_11_2020,550.4,2026.4,3035.4,4185.4,5295.4,5845.4,6120.4,6233.4,6265.4,2020-11-17,6265.4
24_11_2020,642.8,2187.8,3324.8,4559.8,5767.8,6407.8,6748.8,6913.8,6955.8,2020-11-24,6955.8
01_12_2020,695.6,2363.6,3530.6,4839.6,6067.6,6708.6,7104.6,7262.6,7305.6,2020-12-01,7305.6
08_12_2020,1084.8,3607.8,5262.8,7179.8,8914.8,9695.8,10206.8,10401.8,10465.8,2020-12-08,10465.8
15_12_2020,1534.4,6103.4,9018.4,12103.4,15028.4,16519.4,17265.4,17602.4,17707.4,2020-12-15,17707.4
22_12_2020,1560.8,6215.8,9433.8,12896.8,16295.8,18163.8,19342.8,19806.8,19945.8,2020-12-22,19945.8
29_12_2020,943.2,3998.2,6517.2,9209.2,11879.2,13360.2,14384.2,14988.2,15192.2,2020-12-29,15192.2
05_01_2021,701.6,3216.6,5200.6,7113.6,9191.6,10553.6,11499.6,12102.6,12298.6,2021-01-05,12298.6
12_01_2021,548.8,2545.8,3952.8,5378.8,7038.8,8038.8,8727.8,9237.8,9425.8,2021-01-12,9425.8
19_01_2021,287.6,1342.6,2252.6,3246.6,4304.6,4927.6,5354.6,5733.6,5888.6,2021-01-19,5888.6


In [57]:
# curdf[curdf.Age == '20-29']
# dfCase.diff().iloc[:,1:-1]

In [58]:
dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

dfAdmDiff = dfAdmDiff.iloc[:-1]
dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']

# dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

# Load calculations

In [59]:
datafilename = '../model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

In [60]:
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)

In [61]:
curdf = df[df.vacc1 == 0.75]

In [62]:
def cmaptohex(tup):
    (r,g,b,a) = tup
    return "#{:02x}{:02x}{:02x}".format(int(r*255),int(g*255),int(b*255))

In [63]:
curdf[curdf.Age == '10-19']

,Unnamed: 0,Age,CaseHospRate,Population,Start,End,RT,Count,CountCumul,vacc1,PctCumul,HospCumul,RT2
154,154,10-19,0.01,273589,2021-05-18,2021-05-18,0.9,6.860000e+02,0.000000,0.75,0.000000,0.000000,9
155,155,10-19,0.01,273589,2021-05-18,2021-05-25,0.9,6.391477e+02,639.147698,0.75,0.002336,6.391477,9
156,156,10-19,0.01,273589,2021-05-18,2021-06-01,0.9,5.921772e+02,1231.324890,0.75,0.004501,12.313249,9
157,157,10-19,0.01,273589,2021-05-18,2021-06-08,0.9,5.326283e+02,1763.953149,0.75,0.006447,17.639531,9
158,158,10-19,0.01,273589,2021-05-18,2021-06-15,0.9,3.897356e+02,2153.688770,0.75,0.007872,21.536888,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2107,10-19,0.01,273589,2021-05-18,2021-09-14,1.7,6.531782e-05,9700.314098,0.75,0.035456,97.003141,17
2108,2108,10-19,0.01,273589,2021-05-18,2021-09-21,1.7,1.142711e-05,9700.314110,0.75,0.035456,97.003141,17
2109,2109,10-19,0.01,273589,2021-05-18,2021-09-28,1.7,1.999131e-06,9700.314112,0.75,0.035456,97.003141,17
2110,2110,10-19,0.01,273589,2021-05-18,2021-10-05,1.7,3.497405e-07,9700.314112,0.75,0.035456,97.003141,17


# Nye smittetilfælde

In [64]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

casedf = pd.DataFrame()

casedf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
casedf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


casedf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    casedf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 
    curVals = prevVals + newVals
    # casedf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 
    casedf[curAge] = curVals
    prevVals = curVals

    casedf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Count.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        casedf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 
        casedf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Count.values 


# # Remove everythings after september first
# casedf = casedf[casedf.Date <= np.datetime64('2021-09-01')]

# casedf

In [65]:
casedf

,Date,Week,Total,Total_9,Total_10,Total_11,Total_12,Total_13,Total_14,Total_15,...,90+,90+_9,90+_10,90+_11,90+_12,90+_13,90+_14,90+_15,90+_16,90+_17
0,2021-05-18,20,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,5742.000000,...,5742.000000,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
1,2021-05-25,21,6417.999902,5013.857624,5381.395161,5737.029927,6082.182325,6417.999902,6745.426619,7065.250857,...,6417.999902,8.726226e-01,9.365896e-01,9.984850e-01,1.058556e+00,1.117003e+00,1.173989e+00,1.229651e+00,1.284107e+00,1.337455e+00
2,2021-06-01,22,7149.149679,4363.138230,5026.257629,5712.539396,6420.573872,7149.149679,7897.212454,8663.834553,...,7149.149679,1.087458e-01,1.252732e-01,1.423780e-01,1.600248e-01,1.781837e-01,1.968282e-01,2.159353e-01,2.354845e-01,2.554573e-01
3,2021-06-08,23,7911.077371,3775.813770,4667.231832,5653.534757,6734.795493,7911.077371,9182.435724,10548.919136,...,7911.077371,1.355130e-02,1.675515e-02,2.030129e-02,2.419020e-02,2.842229e-02,3.299795e-02,3.791756e-02,4.318144e-02,4.878991e-02
4,2021-06-15,24,8413.934336,3147.864843,4172.794557,5384.119727,6793.940046,8413.934336,10255.400844,12329.290436,...,8413.934336,1.688679e-03,2.240968e-03,2.894687e-03,3.656690e-03,4.533634e-03,5.532006e-03,6.658136e-03,7.918213e-03,9.318299e-03
5,2021-06-22,25,8679.174464,2554.715157,3628.860219,4983.133105,6654.145389,8679.174464,11096.037054,13942.971774,...,8679.174464,2.104325e-04,2.997248e-04,4.127426e-04,5.527597e-04,7.231583e-04,9.274225e-04,1.169134e-03,1.451966e-03,1.779681e-03
6,2021-06-29,26,8755.367797,2038.116440,3099.122675,4523.770779,6383.696702,8755.367797,11719.512683,15360.498523,...,8755.367797,2.622278e-05,4.008756e-05,5.885140e-05,8.355734e-05,1.153507e-04,1.554793e-04,2.052937e-04,2.662474e-04,3.398972e-04
7,2021-07-06,27,8458.000279,1567.726965,2548.872105,3949.010265,5878.061610,8458.000279,11821.301708,16108.851252,...,8458.000279,3.267718e-06,5.361628e-06,8.391398e-06,1.263086e-05,1.839954e-05,2.606558e-05,3.604848e-05,4.882187e-05,6.491618e-05
8,2021-07-13,28,7453.195746,1109.179955,1925.713894,3161.210499,4951.901078,7453.195746,10835.814736,15279.771078,...,7453.195746,4.072024e-07,7.171065e-07,1.196498e-06,1.909330e-06,2.934903e-06,4.369806e-06,6.329920e-06,8.952481e-06,1.239819e-05
9,2021-07-20,29,5868.514492,706.895497,1309.194631,2273.527257,3739.380464,5868.514492,8837.013750,12822.022065,...,5868.514492,5.074300e-08,9.591149e-08,1.706041e-07,2.886218e-07,4.681451e-07,7.325833e-07,1.111500e-06,1.641619e-06,2.367902e-06


In [66]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
curMaxY = 30000
firstDate = np.datetime64('2021-05-18')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("cases.html",title='Nye smittetilfælde')

show(column(plot,slider))

None


In [67]:

# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(casedf)
sourceData = ColumnDataSource(dfCaseDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(casedf.Total_17)
curMaxY = 30000
firstDate = np.datetime64('2021-05-18')
firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye smittetilfælde'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


# testDates = np.arange(np.datetime64('2021-05-01'),np.datetime64('2021-05-30'))
# testData = np.multiply(np.ones((len(testDates),)),5000)
# plot.line(testDates,testData)

minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("casesData.html",title='Nye smittetilfælde')

show(column(plot,slider))

None


# Hospitaliseringer

In [13]:
curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)

<ipython-input-13-7d6623e11856>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curdf['Hosp'] = np.multiply(curdf.Count,curdf.CaseHospRate)


In [14]:

# curdf['x'] = np.arange(0,len(curdf))
# curdf['y'] = np.arange(0,len(curdf))

admdf = pd.DataFrame()

admdf['Date'] = curdf.End.unique().astype('datetime64[D]')
numVals = len(curdf.End.unique())
firstWeek = 20
weekNums = np.arange(firstWeek,firstWeek+numVals)
admdf['Week'] = weekNums

 


allAges = df.Age.unique()

allRTs = df.RT2.unique()


admdf['Total'] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)
prevVals = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)

for curRT in allRTs:
    curColName = 'Total_'+str(curRT)
    admdf[curColName] = np.zeros(curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Hosp.values.shape)



for curAge in allAges:
    newVals = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 
    curVals = prevVals + newVals
    # admdf[curAge] = curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 
    admdf[curAge] = curVals
    prevVals = curVals

    admdf['Total'] += curdf[(curdf.Age == curAge) & (curdf.RT2 == 13)].Hosp.values 

    for curRT in allRTs:
        curColName = curAge+'_'+str(curRT)
        admdf[curColName] = curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 
        admdf['Total_'+str(curRT)] += curdf[(curdf.Age == curAge) & (curdf.RT2 == curRT)].Hosp.values 


# # Remove everythings after september first
# admdf = admdf[admdf.Date <= np.datetime64('2021-09-01')]

admdf

,Date,Week,Total,Total_9,Total_10,Total_11,Total_12,Total_13,Total_14,Total_15,...,90+,90+_9,90+_10,90+_11,90+_12,90+_13,90+_14,90+_15,90+_16,90+_17
0,2021-05-18,20,285.560000,2.855600e+02,285.560000,285.560000,285.560000,285.560000,285.560000,285.560000,...,285.560000,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00,2.800000e+00
1,2021-05-25,21,277.834282,2.170492e+02,232.959813,248.355191,263.296788,277.834282,292.008537,305.853681,...,277.834282,3.490490e-01,3.746358e-01,3.993940e-01,4.234224e-01,4.468010e-01,4.695954e-01,4.918606e-01,5.136429e-01,5.349822e-01
2,2021-06-01,22,279.080766,1.703235e+02,196.209604,222.999929,250.639413,279.080766,308.282831,338.209395,...,279.080766,4.349832e-02,5.010930e-02,5.695118e-02,6.400994e-02,7.127348e-02,7.873129e-02,8.637413e-02,9.419379e-02,1.021829e-01
3,2021-06-08,23,282.825322,1.349611e+02,166.832076,202.097840,240.761378,282.825322,328.292006,377.163517,...,282.825322,5.420519e-03,6.702059e-03,8.120517e-03,9.676079e-03,1.136892e-02,1.319918e-02,1.516702e-02,1.727258e-02,1.951597e-02
4,2021-06-15,24,282.737639,1.057055e+02,140.147025,180.861944,228.260127,282.737639,344.678076,414.453671,...,282.737639,6.754715e-04,8.963871e-04,1.157875e-03,1.462676e-03,1.813454e-03,2.212802e-03,2.663254e-03,3.167285e-03,3.727319e-03
5,2021-06-22,25,282.004500,8.288093e+01,117.771396,161.783938,216.119778,282.004500,360.684164,453.421728,...,282.004500,8.417301e-05,1.198899e-04,1.650970e-04,2.211039e-04,2.892633e-04,3.709690e-04,4.676535e-04,5.807863e-04,7.118725e-04
6,2021-06-29,26,276.104917,6.410023e+01,97.528564,142.454243,201.163403,276.104917,369.874436,485.196900,...,276.104917,1.048911e-05,1.603503e-05,2.354056e-05,3.342293e-05,4.614027e-05,6.219171e-05,8.211747e-05,1.064990e-04,1.359589e-04
7,2021-07-06,27,259.426100,4.787858e+01,77.914273,120.837002,180.068116,259.426100,363.087284,495.531254,...,259.426100,1.307087e-06,2.144651e-06,3.356559e-06,5.052342e-06,7.359816e-06,1.042623e-05,1.441939e-05,1.952875e-05,2.596647e-05
8,2021-07-13,28,225.678953,3.337137e+01,58.012678,95.373283,149.651362,225.678953,328.826387,464.845808,...,225.678953,1.628810e-07,2.868426e-07,4.785991e-07,7.637319e-07,1.173961e-06,1.747922e-06,2.531968e-06,3.580992e-06,4.959276e-06
9,2021-07-20,29,180.948259,2.160705e+01,40.084495,69.748338,114.985752,180.948259,273.344004,398.072744,...,180.948259,2.029720e-08,3.836460e-08,6.824163e-08,1.154487e-07,1.872580e-07,2.930333e-07,4.446001e-07,6.566476e-07,9.471607e-07


In [15]:
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')


dfAdmDiff = dfAdm.diff().iloc[1:]

dfAdmDiff['Dato'] = curDates[1:]

dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

dfAdmDiff = dfAdmDiff.iloc[:-1]
dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]


dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']
dfAdmDiffSum

# admdfData = pd.DataFrame()
# admdfData['Dato'] = curDates[1:]
# admdfData['Total'] = np.diff(dfAdm['I alt'].astype(int).values)
# admdfData = admdfData.iloc[:-1]
# admdfData 

Aldersgruppe,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Dato,I alt
17_11_2020,4,17,35,61,110,165,228,270,284,2020-11-17,284
24_11_2020,1,17,36,64,113,163,222,292,303,2020-11-24,303
01_12_2020,1,8,20,52,97,130,193,252,267,2020-12-01,267
08_12_2020,7,30,62,103,164,231,341,421,442,2020-12-08,442
15_12_2020,14,41,89,148,250,366,486,589,625,2020-12-15,625
22_12_2020,15,57,127,222,365,517,753,938,992,2020-12-22,992
29_12_2020,12,60,110,208,356,526,811,1023,1084,2020-12-29,1084
05_01_2021,14,51,109,181,302,463,674,898,971,2021-01-05,971
12_01_2021,6,39,76,125,233,349,555,766,824,2021-01-12,824
19_01_2021,3,18,50,96,183,280,447,599,656,2021-01-19,656


In [16]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000

firstDate = np.datetime64('2021-05-18')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

# firstDate = np.datetime64('2021-04-01')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



# for k in range(0,len(allAges)):
#     curAge = allAges[k]
#     curColor = cmaptohex(cmap(k))
#     plot.line('Dato', curAge, source=sourceData,color=curColor)
#     plot.circle('Dato', curAge, source=sourceData,color=curColor)

# plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
# plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hosp.html",title='Hospitaliseringer')

show(column(plot,slider))

None


In [17]:

from bokeh.models import DatetimeTickFormatter
# x = [x*0.005 for x in range(0, 201)] 

# source = ColumnDataSource(data=dict(x=x, y=x))
# source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
source = ColumnDataSource(admdf)
sourceData = ColumnDataSource(dfAdmDiffSum)

figW = 1000
figH = 600

curMaxY = 1.1*max(admdf.Total_17)
# curMaxY = 30000

# firstDate = np.datetime64('2021-05-18')
# lastDate = np.datetime64('2021-09-01')
# plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

firstDate = np.datetime64('2021-04-01')
lastDate = np.datetime64('2021-09-01')
plot = figure(plot_width=figW, plot_height=figH, x_axis_type="datetime",x_range = (firstDate,lastDate),y_range = (0,curMaxY),tools=[])

plot.toolbar.logo = None
plot.yaxis.axis_label = 'Nye hospitalindlæggelser'

# x_values_timestamp = [int(el.timestamp()) for el in casedf['Date']]
# x_values_labels = [el.strftime('%d. %B') for el in casedf['Date']]

plot.xaxis.formatter=DatetimeTickFormatter(days="%d. %b",
months="%d. %b",
hours="%d. %b",
minutes="%d. %b")

# plot.xaxis.ticker = x_values_timestamp
# plot.xaxis.major_label_overrides = dict(casedf['Date'].values,x_values_labels))


# plot.y_range.bounds = (0,None)
cmap = plt.cm.get_cmap('terrain',len(allAges)+1)


curAge = allAges[0]
curColor = cmaptohex(cmap(0))
plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label='16-19')
# plot.varea('Date', 0 ,'10-19', source=source, fill_color=curColor,legend_label=curAge)
# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color=curColor)

for k in range(1,len(allAges)):
    curAge = allAges[k]
    prevAge = allAges[k-1]
    curColor = cmaptohex(cmap(k))
    # plot.line('Week', curAge, source=source, line_width=3, line_alpha=0.6, color=curColor)
    
    plot.varea('Date', prevAge, curAge, source=source, fill_color=curColor,legend_label=curAge)


plot.line('Date', 'Total', source=source, line_width=3, line_alpha=0.6,color='black',legend_label='Total')



for k in range(0,len(allAges)):
    curAge = allAges[k]
    curColor = cmaptohex(cmap(k))
    plot.line('Dato', curAge, source=sourceData,color=curColor)
    plot.circle('Dato', curAge, source=sourceData,color=curColor)

plot.circle('Dato', 'I alt', source=sourceData,color='black',legend_label='Data')
plot.line('Dato', 'I alt', source=sourceData, line_width=3, line_alpha=0.6,color='black')


minRT = 0.9
maxRT = 1.7
slider = Slider(start=minRT, end=maxRT, value=1.3, step=.1, title="Effektivt kontakttal (Rt)", width = figW - 200,align='center')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var curRT = Math.round(f*10)
    var y1 = data['10-19']
    var y2 = data['20-29']
    var y3 = data['30-39']
    var y4 = data['40-49']
    var y5 = data['50-59']
    var y6 = data['60-69']
    var y7 = data['70-79']
    var y8 = data['80-89']
    var y9 = data['90+']
    var yT = data['Total']

    const curName1 = '10-19_'+curRT;
    const curName2 = '20-29_'+curRT;
    const curName3 = '30-39_'+curRT;
    const curName4 = '40-49_'+curRT;
    const curName5 = '50-59_'+curRT;
    const curName6 = '60-69_'+curRT;
    const curName7 = '70-79_'+curRT;
    const curName8 = '80-89_'+curRT;
    const curName9 = '90+_'+curRT; 
    const curNameT = 'Total_'+curRT; 

    // Summing up
    for (var i = 0; i < y1.length; i++) {
        y1[i] = data[curName1][i]
        y2[i] = data[curName2][i] + y1[i]
        y3[i] = data[curName3][i] + y2[i]
        y4[i] = data[curName4][i] + y3[i]
        y5[i] = data[curName5][i] + y4[i]
        y6[i] = data[curName6][i] + y5[i]
        y7[i] = data[curName7][i] + y6[i]
        y8[i] = data[curName8][i] + y7[i]
        y9[i] = data[curName9][i] + y8[i]
        yT[i] = data[curNameT][i]
    }
    
    
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)
slider.js_on_change('start', update_curve)

print(plot.y_range.bounds)
 
output_file("hospData.html",title='Hospitaliseringer')

show(column(plot,slider))

None


# OLD

In [18]:

# plot.line('Week', '10-19', source=source, line_width=3, line_alpha=0.6, color='#00ff00')
# plot.line('Week', '20-29', source=source, line_width=3, line_alpha=0.6, color='#20dd20')
# plot.line('Week', '30-39', source=source, line_width=3, line_alpha=0.6, color='#307030')
# plot.line('Week', '40-49', source=source, line_width=3, line_alpha=0.6, color='#406040')
# plot.line('Week', '50-59', source=source, line_width=3, line_alpha=0.6, color='#505050')
# plot.line('Week', '60-69', source=source, line_width=3, line_alpha=0.6, color='#604060')
# plot.line('Week', '70-79', source=source, line_width=3, line_alpha=0.6, color='#703070')
# plot.line('Week', '80-89', source=source, line_width=3, line_alpha=0.6, color='#802080')
# plot.line('Week',   '90+', source=source, line_width=3, line_alpha=0.6, color='#901090')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6,color='black')

# plot.circle('Week', '10-19', source=source, color='#108000')
# plot.circle('Week', '20-29', source=source, color='#208000')
# plot.circle('Week', '30-39', source=source, color='#308000')
# plot.circle('Week', '40-49', source=source, color='#408000')
# plot.circle('Week', '50-59', source=source, color='#508000')
# plot.circle('Week', '60-69', source=source, color='#608000')
# plot.circle('Week', '70-79', source=source, color='#708000')
# plot.circle('Week', '80-89', source=source, color='#808000')
# plot.circle('Week',   '90+', source=source, color='#908000')
# plot.circle('Week', 'Total', source=source,color='black')
# plot.line('Week', 'Total', source=source, line_width=3, line_alpha=0.6)


# plot.varea('Week', 0 ,'10-19', source=source, fill_color=cmaptohex(cmap(1)))
# plot.varea('Week','10-19', '20-29', source=source, color=cmaptohex(cmap(2)))
# plot.varea('Week','20-29', '30-39', source=source, color=cmaptohex(cmap(3)))
# plot.varea('Week','30-39', '40-49', source=source, color=cmaptohex(cmap(4)))
# plot.varea('Week','40-49', '50-59', source=source, color=cmaptohex(cmap(5)))
# plot.varea('Week','50-59', '60-69', source=source, color=cmaptohex(cmap(6)))
# plot.varea('Week','60-69', '70-79', source=source, color=cmaptohex(cmap(7)))
# plot.varea('Week','70-79', '80-89', source=source, color=cmaptohex(cmap(8)))
# plot.varea('Week','80-89',   '90+', source=source, color=cmaptohex(cmap(9)))

# slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

In [19]:
# curdf['RT2'] = np.round(np.multiply(curdf.RT,10)).astype(int)


    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i]
    #     y3[i] = data[curName3][i]
    #     y4[i] = data[curName4][i]
    #     y5[i] = data[curName5][i]
    #     y6[i] = data[curName6][i]
    #     y7[i] = data[curName7][i]
    #     y8[i] = data[curName8][i]
    #     y9[i] = data[curName9][i]
    #     yT[i] = data[curNameT][i]
    # }


    # // Summing up
    # for (var i = 0; i < y1.length; i++) {
    #     y1[i] = data[curName1][i]
    #     y2[i] = data[curName2][i] + y1[i]
    #     y3[i] = data[curName3][i] + y2[i]
    #     y4[i] = data[curName4][i] + y3[i]
    #     y5[i] = data[curName5][i] + y4[i]
    #     y6[i] = data[curName6][i] + y5[i]
    #     y7[i] = data[curName7][i] + y6[i]
    #     y8[i] = data[curName8][i] + y7[i]
    #     y9[i] = data[curName9][i] + y8[i]
    #     yT[i] = data[curNameT][i]
    # }

In [20]:

# # x = [x*0.005 for x in range(0, 201)] 

# # source = ColumnDataSource(data=dict(x=x, y=x))
# # source = ColumnDataSource(data=dict(x=curdf.End, y=curdf.Count, RT09=curdf[curdf.RT == 0.9].Count))
# source = ColumnDataSource(curdf)

# plot = figure(plot_width=400, plot_height=400)
# plot.line('x', 'Count', source=source, line_width=3, line_alpha=0.6)

# # slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")
# minRT = 0.9
# maxRT = 1.7
# slider = Slider(start=minRT, end=maxRT, value=1.1, step=.1, title="RT")

# update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
#     var data = source.data;
#     var f = slider.value;
#     var age = data['Age']
#     var RT = data['RT2']
#     var End = data['End']
#     var Count = data['Count']

#     var x = data['x']
#     var y = data['y']

#     console.log(Count)
#     for (var i = 0; i < RT.length; i++) {
#         y[i] = 6
#         Count[i] = 0
        
#         if (RT[i] == Math.round(f*10)){
#             y[i] = data['Count'][i]
#             console.log(data['Count'][i])
#             Count[i] = data['Count'][i]

#             // End.push(data['End'][i])
#             // Count.push(data['Count'][i])
#         }

#     }

#     //for (var i = 0; i < x.length; i++) {
#         // y[i] = Math.pow(x[i], f)
#     //}
    
#     // necessary becasue we mutated source.data in-place
#     source.change.emit();
# """)
# slider.js_on_change('value', update_curve)

# output_file("plot.html")

# show(column(slider, plot))

In [21]:

# testdf['Total'] = curdf[(curdf.RT2 == 11)].Count.values 
# for curRT in allRTs:
#     curColName = 'Total_'+str(curRT)
#     testdf[curColName] = curdf[(curdf.RT2 == curRT)].Count.values 

# testdf['20_RT10'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 10)].Count.values
# testdf['20_RT11'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['20_RT12'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['20_RT13'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['20_RT14'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values
# testdf['RT11_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 11)].Count.values
# testdf['RT12_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 12)].Count.values
# testdf['RT13_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 13)].Count.values
# testdf['RT14_20'] = curdf[(curdf.Age == '20-29') & (curdf.RT2 == 14)].Count.values

    # if (Math.round(f*10) == 11){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT11_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 12){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT12_20'][i]
    #     }
    # }
    # if (Math.round(f*10) == 13){
    #     for (var i = 0; i < y.length; i++) {
    #         y[i] = data['RT13_20'][i]
    #     }
    # }
    
